In [1]:
import ctypes
ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)
# ctypes.windll.kernel32.SetThreadExecutionState(0x80000000)


-2147483648

In [4]:
import os
import grid2op
from stable_baselines3 import PPO, DQN
from grid2op.Runner import Runner
from agent_wrapper import Grid2opAgentWrapper
# from env_wrapper import Grid2opEnvWrapper
from env_wrapper_custom import Grid2opEnvWrapper
from stable_baselines3.common.env_util import make_vec_env
from grid2op.Reward import EpisodeDurationReward, L2RPNReward

env_name = "l2rpn_case14_sandbox"
base_path = os.path.expanduser("C:/Users/henri/data_grid2op")
train_dir = os.path.join(base_path, f"{env_name}_train")
val_dir = os.path.join(base_path, f"{env_name}_val")
test_dir = os.path.join(base_path, f"{env_name}_test")

def directories_exist(train_path, val_path, test_path):
    return os.path.exists(train_path) and os.path.exists(val_path) and os.path.exists(test_path)

# Make environment
env = grid2op.make(env_name, reward_class=EpisodeDurationReward)

if directories_exist(train_dir, val_dir, test_dir):
    print("Directories already exist. Skipping train-val-test split.")
else:
    print("Directories do not exist. Proceeding with train-val-test split.")
    # Perform the split
    try:
        nm_env_train, nm_env_val, nm_env_test = env.train_val_split_random(
            pct_val=10, # 10% validation
            pct_test=10, # 10% test
            add_for_train="train",
            add_for_val="val",
            add_for_test="test"
        )
        print(f"Training environment created: {nm_env_train}")
        print(f"Validation environment created: {nm_env_val}")
        print(f"Test environment created: {nm_env_test}")
    except OSError as e:
        print(f"An error occurred during splitting: {e}")


Directories already exist. Skipping train-val-test split.


In [6]:
log_dir = "./tensorboard_logs/"

tb_log_name = "PPO_default_Grid2op"
model_path = "./PPO_default_sandbox"

env_config = {
    # "backend_cls": ,
    # "backend_options": {},
    "env_name": "l2rpn_case14_sandbox", # "l2rpn_neurips_2020_track1_small"
    "env_is_test": False,
    "obs_attr_to_keep": ["gen_p", "p_or" ,"load_p", "rho", "line_status"], # "gen_q", "gen_v",
    "act_type": "discrete", # "discrete" "box" "multi_discrete"
    "act_attr_to_keep": ["change_line_status", "set_line_status_simple", "set_bus"], # set_line_status
    "reward_class": EpisodeDurationReward,
    "data_set": "train", # for training data set train/val/test
}


vec_env = make_vec_env(lambda: Grid2opEnvWrapper(env_config), n_envs=1)

#sb3_algo2 = PPO.load(model_path, env=vec_env)
#print("Model loaded successfully")


custom_hyperparameters = {
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 64,
    "n_epochs": 10,
    "gamma": 0.99,
    "gae_lambda": 0.95,
    "clip_range": 0.2,
    "clip_range_vf": None,
    "normalize_advantage": True,
    "ent_coef": 0.0,
    "vf_coef": 0.5,
    "max_grad_norm": 0.5,
    "use_sde": False,
    "sde_sample_freq": -1,
    "target_kl": None,
}

# Initialize PPO with custom hyperparameters
sb3_algo = PPO(
    "MlpPolicy",
    vec_env,
    verbose=0,
    tensorboard_log="./tensorboard_logs/",
    **custom_hyperparameters
)



sb3_algo.learn(total_timesteps=1_000_000, tb_log_name=tb_log_name)

# Save the model
sb3_algo.save(model_path)

print(f"Model saved at {model_path}")



Model saved at ./PPO_default_sandbox


In [38]:
from agent_wrapper import Grid2opAgentWrapper
from grid2op.Runner import Runner

nb_episode_test = 2
seeds_test_env = (0, 1)    # same size as nb_episode_test
seeds_test_agent = (3, 4)  # same size as nb_episode_test
ts_ep_test =  (0, 1)       # same size as nb_episode_test

# uncomment those two line if the env_config is the correct for the tested agent
# test_env_config = env_config
# test_env_config["data_set"] = "val"


test_env_config = {
    # "backend_cls": ,
    # "backend_options": {},
    "env_name": "l2rpn_case14_sandbox", # "l2rpn_neurips_2020_track1_small"
    "env_is_test": False,
    # "obs_attr_to_keep": ["gen_p", "p_or" ,"load_p", "rho", "line_status"], # "gen_q", "gen_v",
    "act_type": "discrete", # "discrete" "box" "multi_discrete"
    # "act_attr_to_keep": ["change_line_status", "set_line_status_simple", "set_bus"], # set_line_status
    "reward_class": EpisodeDurationReward,
    "data_set": "val", # for training data set train/val/test
}


# test_vec_env = make_vec_env(lambda: Grid2opEnvWrapper(test_env_config), n_envs=1)

test_env = Grid2opEnvWrapper(test_env_config)

sb3_algo_to_test = PPO.load(model_path, env=test_env)

my_agent = Grid2opAgentWrapper(test_env, sb3_algo_to_test)
runner = Runner(**test_env._g2op_env.get_params_for_runner(),
                agentClass=None,
                agentInstance=my_agent)

res = runner.run(nb_episode=nb_episode_test,
                 env_seeds=seeds_test_env,
                 agent_seeds=seeds_test_agent,
                 episode_id=ts_ep_test,
                 add_detailed_output=True,
                 path_save=model_path+"_val"
                 )

In [39]:
for ep_data in res:
    chron_name, cum_reward, nb_time_step, max_ts, _, ep_data = ep_data
    print(f"Episode {i + 1}:")
    print(f"  Chronics Name: {chron_name}")
    print(f"  Cumulative Reward: {cum_reward}")
    print(f"  Number of Time Steps: {nb_time_step}")
    print(f"  Maximum Time Steps: {max_ts}")
    print("  Detailed Data:")
    for t, obs in enumerate(ep_data.observations):
        print(f"    Time Step {t}:")
        print(f"      Observation: {obs}")
        print(f"      Action Taken: {ep_data.actions[t]}")
        print(f"      Reward: {ep_data.rewards[t]}")
        break
    break

Episode 1:
  Chronics Name: C:\Users\henri\data_grid2op\l2rpn_case14_sandbox_val\chronics\0021
  Cumulative Reward: 0021
  Number of Time Steps: 0.13578869047619047
  Maximum Time Steps: 1095
  Detailed Data:
    Time Step 0:
      Observation: <grid2op.Space.GridObjects.CompleteObservation_l2rpn_case14_sandbox_val object at 0x00000246A7699390>
      Action Taken: This action will:
	 - NOT change anything to the injections
	 - NOT perform any redispatching action
	 - NOT modify any storage capacity
	 - NOT perform any curtailment
	 - NOT force any line status
	 - NOT switch any line status
	 - NOT switch anything in the topology
	 - Set the bus of the following element(s):
	 	 - Assign bus 1 to line (origin) id 7 [on substation 5]
	 	 - Assign bus 1 to line (origin) id 8 [on substation 5]
	 	 - Assign bus 1 to line (origin) id 9 [on substation 5]
	 	 - Assign bus 1 to line (extremity) id 17 [on substation 5]
	 	 - Assign bus 1 to generator id 2 [on substation 5]
	 	 - Assign bus 1 to g